In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Task : 
1. Create a decision tree for the dataset
2. Create confusion matrix, Gini Impurity etc
3. Perform prediction & calculate accuracy
4. Build the same program using built-in lib

# 1. Create a decision tree for the dataset

1.1 read the csv

In [2]:
# df891 = pd.read_csv('../input/titanic893/titanic_893.csv') # kaggle

df891 = pd.read_csv(r'S:\SandeepG\Official\DataScience\gitHub\data\kaggle\titanic\titanic_893.csv') # local

In [3]:
df891.shape

(891, 12)

In [4]:
df891.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
pd.set_option("display.max_rows", None)
print(df891['Fare'].value_counts(bins = 10).sort_index())
pd.set_option("display.max_rows", 20)

(-0.513, 51.233]      732
(51.233, 102.466]     106
(102.466, 153.699]     31
(153.699, 204.932]      2
(204.932, 256.165]     11
(256.165, 307.398]      6
(307.398, 358.63]       0
(358.63, 409.863]       0
(409.863, 461.096]      0
(461.096, 512.329]      3
Name: Fare, dtype: int64


In [7]:
pd.set_option("display.max_rows", None)
print(df891['Age'].value_counts(bins = 10).sort_index())
pd.set_option("display.max_rows", 20)

(0.339, 8.378]       54
(8.378, 16.336]      46
(16.336, 24.294]    177
(24.294, 32.252]    169
(32.252, 40.21]     118
(40.21, 48.168]      70
(48.168, 56.126]     45
(56.126, 64.084]     24
(64.084, 72.042]      9
(72.042, 80.0]        2
Name: Age, dtype: int64


# 2. Create confusion matrix, Gini Impurity etc

In [30]:

def cal_gini(p_df, p_label, p_unique_threshold = 5):

    df_return = pd.DataFrame(columns = ['ColName', 'ColValue', 'survived', 'notSurvived', 'gini'], dtype = object)


    for currColName in p_df.columns:
    
        if currColName == p_label: # skip if the column is target label
            continue
    
        if p_df[currColName].nunique() > p_unique_threshold: # skip considering it root node if distinct value is more than 9
            continue
        
        df_curr_col = df891[['PassengerId', currColName, 'Survived']]
        
        for currColValue in df_curr_col[currColName].unique():
                       
            survived = int(df_curr_col[(df_curr_col[currColName] == currColValue) & (df_curr_col.Survived == 1)].PassengerId.agg(['count']))
            notSurvived = int(df_curr_col[(df_curr_col[currColName] == currColValue) & (df_curr_col.Survived == 0)].PassengerId.agg(['count']))
            total = survived + notSurvived
            
            gini = None #reset 
            if total != 0:
                gini = 1 - ((survived/total)**2) - ((notSurvived/total)**2)
            
            #print(f'currColName : {currColName} currColValue : {currColValue} survived : {survived} notSurvived : {notSurvived} gini : {gini}')
            df_return = df_return.append({'ColName' : currColName, 'ColValue' : currColValue, 'survived' : survived, 'notSurvived' : notSurvived, 'gini' : gini},ignore_index = True)
            
    return df_return



        #break
            
        #survived = int(df_curr_col[(df_curr_col.Survived == 1)].PassengerId.agg(['count']))
        #notSurvived = int(df_curr_col[(df_curr_col.Survived == 0)].PassengerId.agg(['count']))
        
        
        #print(f'col_name : {col_name} \tvalue_counts : {df_curr_col[col_name].nunique()} \t df_curr_col.shape : {df_curr_col.columns} survived : {survived} notSurvived : {notSurvived} gini : {gini}')

        

In [31]:
myRootFeature = cal_gini(p_df = df891, p_label = 'Survived', p_unique_threshold = 3)
myRootFeature.sort_values(by=['gini'])

,ColName,ColValue,survived,notSurvived,gini
3,Sex,male,109,468,0.306444
0,Pclass,3,119,372,0.367246
4,Sex,female,233,81,0.382835
5,Embarked,S,217,427,0.446834
1,Pclass,1,136,80,0.466392
7,Embarked,Q,30,47,0.475628
6,Embarked,C,93,75,0.49426
2,Pclass,2,87,97,0.498523
8,Embarked,NaN,0,0,None


In [33]:
df_root = pd.DataFrame(df891)

df_root['branch1_SexEqMale'] = df_root["Sex"].apply(lambda x: 1 if x == 'male' else 0)
df_root['branch2_SexNEqMale'] = df_root["Sex"].apply(lambda x: 1 if x == 'female' else 0)

#df_root['branch11_PclassEq3'] = df_root["Pclass"].apply(lambda x: 1 if (x == 3) else 0)
df_root['branch11_PclassEq3'] = df_root.apply(lambda x: 1 if x['branch1_SexEqMale'] == 1 and x['Pclass'] == 3 else 0, axis = 1)
df_root['branch12_PclassNEq3'] = df_root.apply(lambda x: 1 if x['branch1_SexEqMale'] == 1 and x['Pclass'] != 3 else 0, axis = 1)

df_root['branch111_EmbarkedEqS'] = df_root.apply(lambda x: 1 if x['branch11_PclassEq3'] == 1 and x['Embarked'] == 'S' else 0, axis = 1)
df_root['branch112_EmbarkedNEqS'] = df_root.apply(lambda x: 1 if x['branch11_PclassEq3'] == 1 and x['Embarked'] != 'S' else 0, axis = 1)

for col_name in df_root.filter(regex='^branch',axis=1).columns: 
    col_count = int(df_root[(df_root[col_name] == 1)].PassengerId.agg(['count']))
    survived = int(df_root[(df_root[col_name] == 1) & (df_root.Survived == 1)].PassengerId.agg(['count']))
    notSurvived = int(df_root[(df_root[col_name] == 1) & (df_root.Survived == 0)].PassengerId.agg(['count']))
    print(f'col_name : {col_name} col_count : {col_count} survived : {survived} notSurvived : {notSurvived} Percent : {(notSurvived/(survived + notSurvived))*100}')
    

col_name : branch1_SexEqMale col_count : 577 survived : 109 notSurvived : 468 Percent : 81.10918544194108
col_name : branch2_SexNEqMale col_count : 314 survived : 233 notSurvived : 81 Percent : 25.796178343949045
col_name : branch11_PclassEq3 col_count : 347 survived : 47 notSurvived : 300 Percent : 86.45533141210375
col_name : branch12_PclassNEq3 col_count : 230 survived : 62 notSurvived : 168 Percent : 73.04347826086956
col_name : branch111_EmbarkedEqS col_count : 265 survived : 34 notSurvived : 231 Percent : 87.16981132075472
col_name : branch112_EmbarkedNEqS col_count : 82 survived : 13 notSurvived : 69 Percent : 84.14634146341463


# 3. Perform prediction & calculate accuracy

Performing prediction for Male passenger only

In [36]:
# our analysis says that Passenger = Male & Pclass = 3 has 86% chances of not surviving

# for our prediction, ,we will discard any row which is not male

for i in range(0,10):
    
    df_prediction = None
    correctPredCount = None
    inCorrectPredCount = None
    accuracy = None
    maxPassengerId = None
    minPassengerId = None
    
    r = random.randint(70, 100)
    
    df_test = df_root[(df_root['Sex'] == 'male')].sample(n = r)

    df_prediction = df_test[['PassengerId', 'Sex', 'Pclass', 'Survived']]

    df_prediction['Survived_Pred'] = df_prediction.apply(lambda x: 0 if x['Sex'] == 'male' and x['Pclass'] == 3 else 1, axis = 1)
    df_prediction['Pred_Match'] = df_prediction.apply(lambda x: 1 if x['Survived'] == x['Survived_Pred'] else 0, axis = 1)

    maxPassengerId = df_prediction['PassengerId'].max()
    minPassengerId = df_prediction['PassengerId'].min()
    correctPredCount = df_prediction[(df_prediction['Pred_Match'] == 1)].PassengerId.count()
    inCorrectPredCount = df_prediction[(df_prediction['Pred_Match'] == 0)].PassengerId.count()

    accuracy = int(((correctPredCount/(correctPredCount + inCorrectPredCount))) * 100)

    print(f'Iteration : {i}. Sample Size : {r} MaxPassengerId : {maxPassengerId} MinPassengerId : {minPassengerId} \tMy Decision Tress Accuracy : {accuracy} %. ')




C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Iteration : 0. Sample Size : 84 MaxPassengerId : 890 MinPassengerId : 1 	My Decision Tress Accuracy : 61 %. 
Iteration : 1. Sample Size : 82 MaxPassengerId : 887 MinPassengerId : 37 	My Decision Tress Accuracy : 60 %. 
Iteration : 2. Sample Size : 84 MaxPassengerId : 865 MinPassengerId : 18 	My Decision Tress Accuracy : 64 %. 
Iteration : 3. Sample Size : 71 MaxPassengerId : 891 MinPassengerId : 30 	My Decision Tress Accuracy : 69 %. 
Iteration : 4. Sample Size : 70 MaxPassengerId : 890 MinPassengerId : 5 	My Decision Tress Accuracy : 67 %. 
Iteration : 5. Sample Size : 88 MaxPassengerId : 891 MinPassengerId : 1 	My Decision Tress Accuracy : 72 %. 
Iteration : 6. Sample Size : 81 MaxPassengerId : 887 MinPassengerId : 14 	My Decision Tress Accuracy : 58 %. 
Iteration : 7. Sample Size : 99 MaxPassengerId : 870 MinPassengerId : 21 	My Decision Tress Accuracy : 62 %. 
Iteration : 8. Sample Size : 86 MaxPassengerId : 884 MinPassengerId : 1 	My Decision Tress Accuracy : 65 %. 
Iteration : 9.

# ******************************* End Of Program -- Thank You -- ***********************